# HuggingFace

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [2]:
import os
os.environ['HF_TOKEN'] = user_secrets.get_secret("kaggle_hf_key")

In [3]:
from huggingface_hub import whoami

user = whoami(token=user_secrets.get_secret("kaggle_hf_key"))

In [4]:
from transformers import pipeline
import torch

pipeline = pipeline(task="text-generation", model="mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.bfloat16, device_map="auto")
prompt = """Classify the news into fake or true.
Title: "Scientists develop AI that can predict future trends accurately.",
Content:
A groundbreaking innovation in artificial intelligence has been reported.
The system claims to predict global trends in finance, healthcare, and technology with high accuracy.
"""

outputs = pipeline(prompt, max_new_tokens=100)
for output in outputs:
    print(f"Result: {output['generated_text']}")

2025-05-07 14:09:17.539709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746626957.794469      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746626957.867253      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Classify the news into fake or true.
Title: "Scientists develop AI that can predict future trends accurately.",
Content:
A groundbreaking innovation in artificial intelligence has been reported.
The system claims to predict global trends in finance, healthcare, and technology with high accuracy.
The AI system is based on machine learning algorithms and is designed to analyze large amounts of data.
It uses this data to identify patterns and make predictions about future trends.
The system has been tested and has shown to be highly accurate in its predictions.

Classification: True.


# Langchain

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("LANGSMITH_API_KEY")
secret_value_1 = user_secrets.get_secret("mistral_legalBot")

In [6]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = user_secrets.get_secret("LANGSMITH_API_KEY")
    
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = "langchain kaggle"

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = user_secrets.get_secret("mistral_legalBot")


In [7]:
!pip install -U langchain-mistralai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35


In [8]:
from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

## Unrestricted Prompt

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Classify the news into fake or true"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "Title: {title} Content: {content}")]
)

In [10]:
title = "Aliens spotted in the White House"
text = "Eyewitnesses report that unknown beings were seen entering the West Wing late last night."

In [11]:
prompt = prompt_template.invoke({"title": title, "content": text})

In [12]:
response = model.invoke(prompt)
print(response.content)

Based on the content provided, the news "Aliens spotted in the White House" is highly likely to be fake. Here are a few reasons for this classification:

1. **Extraordinary Claim**: The claim that aliens were spotted in the White House is extraordinary and would require extraordinary evidence to be credible.
2. **Lack of Credible Sources**: The news does not mention any credible sources or officials confirming the incident.
3. **Lack of Mainstream Coverage**: Such a significant event would be widely covered by mainstream media outlets. The lack of such coverage suggests that the news is likely fake.
4. **Vague Details**: The content provides vague details, such as "unknown beings" and "eyewitnesses report," without specifying who the eyewitnesses are or providing any concrete descriptions of the beings.

Without further evidence or credible sources, it's reasonable to classify this news as fake.


## Few Shot Prompt

In [13]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Title: {Title}\nContent: {Content}\nAnswer: {answer}")

In [14]:
examples = [
    {
        "Title": "UN Urges Global Action on Climate Change as Sea Levels Rise",
        "Content": """
The United Nations has called on world leaders to commit to stronger climate policies, warning that rising sea levels pose an imminent threat to coastal cities.
Recent satellite data shows that ocean levels have risen by more than 8 inches since 1880.
""",
        "answer":"True"
    },
    {
        "Title": "Apple Announces New MacBook with AI-Enhanced Processor",
        "Content": """
Apple unveiled its latest MacBook Pro featuring the M4 chip, designed with advanced AI processing capabilities.
The company says it offers 30% better performance and improved battery life compared to the previous generation.
""",
        "answer":"True"
    },
    {
        "Title": "NASA Confirms Moon is Made Entirely of Cheese",
        "Content": """
In a shocking revelation, a leaked report from NASA claims that the Moon's surface is composed mostly of cheddar cheese.
Scientists are baffled and plan to launch a new mission to collect samples for pizza experiments.
""",
        "answer":"Fake"
    },
    {
        "Title": "Drinking Coffee Turns You Invisible, New Study Finds",
        "Content": """
A new 'study' by researchers at an unverified institution suggests that drinking six cups of coffee per day can make the human body temporarily invisible.
Experts are skeptical, citing a lack of peer-reviewed evidence.
""",
        "answer":"Fake"
    },
]


In [15]:
print(example_prompt.invoke(examples[0]).to_string())

Title: UN Urges Global Action on Climate Change as Sea Levels Rise
Content: 
The United Nations has called on world leaders to commit to stronger climate policies, warning that rising sea levels pose an imminent threat to coastal cities.
Recent satellite data shows that ocean levels have risen by more than 8 inches since 1880.

Answer: True


In [16]:
from langchain_core.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Title: {Title}\nContent: {Content}\nAnswer:",
    input_variables=["Title", "Content"],
)

In [17]:
query = {
    "Title": "Scientists develop AI that can predict future trends accurately.",
    "Content": """
A groundbreaking innovation in artificial intelligence has been reported.
The system claims to predict global trends in finance, healthcare, and technology with high accuracy.
"""
}

few_shot_prompt_output = few_shot_prompt.format(**query)


In [18]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Classify the news into fake or true"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", few_shot_prompt_output)]
)
final_prompt = prompt_template.format()

In [19]:
print(final_prompt)

System: Classify the news into fake or true
Human: Title: UN Urges Global Action on Climate Change as Sea Levels Rise
Content: 
The United Nations has called on world leaders to commit to stronger climate policies, warning that rising sea levels pose an imminent threat to coastal cities.
Recent satellite data shows that ocean levels have risen by more than 8 inches since 1880.

Answer: True

Title: Apple Announces New MacBook with AI-Enhanced Processor
Content: 
Apple unveiled its latest MacBook Pro featuring the M4 chip, designed with advanced AI processing capabilities.
The company says it offers 30% better performance and improved battery life compared to the previous generation.

Answer: True

Title: NASA Confirms Moon is Made Entirely of Cheese
Content: 
In a shocking revelation, a leaked report from NASA claims that the Moon's surface is composed mostly of cheddar cheese.
Scientists are baffled and plan to launch a new mission to collect samples for pizza experiments.

Answer: Fa

In [20]:
response=model.invoke(final_prompt)

In [21]:
print(response.content)

To determine whether the news is fake or true, let's analyze the content:

Title: Scientists develop AI that can predict future trends accurately.
Content:
A groundbreaking innovation in artificial intelligence has been reported.
The system claims to predict global trends in finance, healthcare, and technology with high accuracy.

Answer: **Potentially True**

Explanation: While the development of AI systems capable of predicting future trends is plausible and aligns with current technological advancements, the claim of "high accuracy" should be approached with caution. Without specific evidence or peer-reviewed studies to back up the claim, it remains potentially true but should be verified through credible sources.


## Few shot chat template

In [22]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "Title": "UN Urges Global Action on Climate Change as Sea Levels Rise",
        "Content": """
The United Nations has called on world leaders to commit to stronger climate policies, warning that rising sea levels pose an imminent threat to coastal cities.
Recent satellite data shows that ocean levels have risen by more than 8 inches since 1880.
""",
        "Classification":"True"
    },
    {
        "Title": "Apple Announces New MacBook with AI-Enhanced Processor",
        "Content": """
Apple unveiled its latest MacBook Pro featuring the M4 chip, designed with advanced AI processing capabilities.
The company says it offers 30% better performance and improved battery life compared to the previous generation.
""",
        "Classification":"True"
    },
    {
        "Title": "NASA Confirms Moon is Made Entirely of Cheese",
        "Content": """
In a shocking revelation, a leaked report from NASA claims that the Moon's surface is composed mostly of cheddar cheese.
Scientists are baffled and plan to launch a new mission to collect samples for pizza experiments.
""",
        "Classification":"Fake"
    },
    {
        "Title": "Drinking Coffee Turns You Invisible, New Study Finds",
        "Content": """
A new 'study' by researchers at an unverified institution suggests that drinking six cups of coffee per day can make the human body temporarily invisible.
Experts are skeptical, citing a lack of peer-reviewed evidence.
""",
        "Classification":"Fake"
    },
]


In [23]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Title: {Title}\nContent: {Content}"),
        ("ai", "Classification: {Classification}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='Title: UN Urges Global Action on Climate Change as Sea Levels Rise\nContent: \nThe United Nations has called on world leaders to commit to stronger climate policies, warning that rising sea levels pose an imminent threat to coastal cities.\nRecent satellite data shows that ocean levels have risen by more than 8 inches since 1880.\n', additional_kwargs={}, response_metadata={}), AIMessage(content='Classification: True', additional_kwargs={}, response_metadata={}), HumanMessage(content='Title: Apple Announces New MacBook with AI-Enhanced Processor\nContent: \nApple unveiled its latest MacBook Pro featuring the M4 chip, designed with advanced AI processing capabilities.\nThe company says it offers 30% better performance and improved battery life compared to the previous generation.\n', additional_kwargs={}, response_metadata={}), AIMessage(content='Classification: True', additional_kwargs={}, response_metadata={}), HumanMessage(content="Title: NASA Confirms Moon is 

In [24]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Classify the news into fake or true."),
        few_shot_prompt,
        ("human", "Title: {Title}\nContent: {Content}"),
    ]
)

In [25]:
chain = final_prompt | model
query = {
    "Title": "Scientists develop AI that can predict future trends accurately.",
    "Content": """
A groundbreaking innovation in artificial intelligence has been reported.
The system claims to predict global trends in finance, healthcare, and technology with high accuracy.
"""
}

response = chain.invoke(query)

In [26]:
print(response.content)

To classify this news as fake or true, let's consider a few points:

1. **Plausibility**: The development of AI systems for predictive analytics is indeed a real and active area of research. Many companies and research institutions are working on such technologies.
2. **Specificity**: The claim mentions "high accuracy" without specifying what that means or providing any data to back it up. This lack of detail is a red flag.
3. **Source**: There is no mention of a specific source or institution that developed this AI. Credible news articles usually cite the source of the information.
4. **Evidence**: No evidence or studies are referenced to support the claim.

Based on these points, the news item seems suspicious but not definitively fake. It falls more into the category of "unverified" or "misleading" rather than outright false. However, without more specific information or a credible source, it's reasonable to be skeptical.

Classification: Likely Fake (or at least Misleading)
